In [1]:
"""Goal : Create a single .json using several .json that will look like that :

      dataset : [{
        "filename" : <REQUIRED: path to the image file>,
        "id" : <REQUIRED: id of the image>,
        "format" : <>
        "tags"
        "class" : {
          "label" : <[0, num_classes)>,
          "text" : <text description of class>
        },
        "object" : {
          "bbox" : {
            "xmin" : [],
            "xmax" : [],
            "ymin" : [],
            "ymax" : [],
            "label" : []
            }
          }
        }
        .
        .
        .
        {
      
        }
      ]
    
Some rules :
Your positive image filename must start with monobloc
No points in a filename (except the one for the extension)
"""


'Goal : Create a single .json using several .json that will look like that :\n\n      dataset : [{\n        "filename" : <REQUIRED: path to the image file>,\n        "id" : <REQUIRED: id of the image>,\n        "format" : <>\n        "tags"\n        "class" : {\n          "label" : <[0, num_classes)>,\n          "text" : <text description of class>\n        },\n        "object" : {\n          "bbox" : {\n            "xmin" : [],\n            "xmax" : [],\n            "ymin" : [],\n            "ymax" : [],\n            "label" : []\n            }\n          }\n        }\n        .\n        .\n        .\n        {\n      \n        }\n      ]\n    \nSome rules :\nYour positive image filename must start with monobloc\nNo points in a filename (except the one for the extension)\n'

In [2]:
%matplotlib inline
import random
import time
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import tensorflow as tf
from pylab import rcParams
import os
import json



In [3]:
"""
Useful functions to read every path in a folder that contain a folder "img" and "ann"
"""

def ReadPaths(paths):
    all_paths = []
    for path in paths:
        temp_paths = ReadDirFiles(path)
        all_paths.extend(temp_paths)
    return all_paths

def ReadDirFiles(dname):
    paths = []
    for file in os.listdir(os.path.join(dname, "img")):
        if not(file.startswith(".ipynb")):
            bname = os.path.basename(file).split(".")[0]

            img_name = os.path.join(dname, "img", file)
            ann_name = os.path.join(dname, "ann", bname + ".json")
            paths.append((img_name, ann_name))
    return paths

In [4]:
"""
function to separate data into training set / validation set / test set and shuffle them
"""

def train_test_split(paths, train_rate=0.8):
    train_paths = []
    valid_paths=[]
    test_paths = []
    
    indices = np.arange(len(paths))
    np.random.shuffle(indices)
    
    nTrain = int(len(paths) * train_rate)
    nValid = (len(paths) - nTrain) // 2
    nTest = len(paths) - nTrain - nValid
    count=0
    
    for path in paths:
        if (indices[count]<nTrain):
            train_paths.append(path)
        else :
            if (indices[count]<nValid+nTrain):
                valid_paths.append(path)
            else:
                test_paths.append(path)
        count+=1
    
    #let's shuffle a bit 
    random.shuffle(train_paths)
    random.shuffle(valid_paths)
    random.shuffle(test_paths)
    
    return train_paths, valid_paths, test_paths

In [5]:
"""
A .json created by Supervisely looks like that :
{"description": "", 
    "tags": ["positive"], 
    "size": {"height": 480, "width": 480}, 
    "objects": [{"description": "", "bitmap": null, "tags": [], "classTitle": "monobloc_bbox", 
    "points": {"exterior": [[310, 124], [399, 260]], "interior": []}
},
.
.
.
{
    another box
}

exterior points out to our bounding box : The two first number are coordinates of the top left corner of the box,
the second is the bottom right.
The vertical/horizontal flip doesn't work on Supervisely. 
Sometimes, Supervisely can switch those points, that's why we will correct if there is a problem

"""
def LoadBbox(data):
    left=[]
    top=[]
    right=[]
    bottom=[]
    count=0
    text=[]
    
    height, width = data["size"]["height"],  data["size"]["width"]

    for bbox in data["objects"]:
        
        #We need to rectify flip_vert and flip_hori
        if "flip_vert" in data["tags"]:
            bbox["points"]["exterior"][0][0]=width/2-(bbox["points"]["exterior"][0][0]-width/2) #x-axis top left
            bbox["points"]["exterior"][1][0]=width/2-(bbox["points"]["exterior"][1][0]-width/2) #x-axis bottom right 
        if "flip_hori" in data["tags"]:
            bbox["points"]["exterior"][0][1]=height/2-(bbox["points"]["exterior"][0][1]-height/2) #y-axis top left
            bbox["points"]["exterior"][1][1]=height/2-(bbox["points"]["exterior"][1][1]-height/2) #y-axis bottom right
          
        
        
        #We need to rectify some errors from Supervisely and to normalize coordinates.
        #The origin of the landmark is the top left corner of the image, so bottom > top and right > left
        if (bbox["points"]["exterior"][0][0]<bbox["points"]["exterior"][1][0]): #good case
            left.append(bbox["points"]["exterior"][0][0]/width) #x-axis top left
            right.append(bbox["points"]["exterior"][1][0]/width) #x-axis bottom right 
        else : #bad case, we need to switch points
            left.append(bbox["points"]["exterior"][1][0]/width)
            right.append(bbox["points"]["exterior"][0][0]/width)
        if (bbox["points"]["exterior"][0][1]<bbox["points"]["exterior"][1][1]): #good case
            top.append(bbox["points"]["exterior"][0][1]/height) #y-axis top left
            bottom.append(bbox["points"]["exterior"][1][1]/height) #y-axis bottom right
        else : #bad case, we need to switch points
            top.append(bbox["points"]["exterior"][1][1]/height)
            bottom.append(bbox["points"]["exterior"][0][1]/height)
        count+=1
    
    return count, left, right, top, bottom

In [6]:
def create_json(paths,name,drop_neg_rate=0.0):
    #We initialize the dictionnary which will be saved
    final_json=[] 
    
    
    #We initialize the format of a .json representing a single image
    data={"filename":"","id":0,"height":0,"width":0,"format":"","tags":"",
          "object":{"bbox":{},"count":0}}
    current_id=0
    count_pos=0
    
    for path in paths:

        img_path=path[0]
        ann_path=path[1]
        print([img_path, ann_path])
        
        #We deal with positive images
        if (img_path.startswith("Images+json/img/monobloc")):
            with open(ann_path, 'r') as f:
                jload = json.load(f)
            data["filename"]=img_path
            data["id"]=current_id
            data["height"]=jload["size"]["height"]
            data["width"]=jload["size"]["width"]
            data["format"]=img_path.split(".")[1]
            #we load bounding boxes
            count, xmin, xmax, ymin, ymax = LoadBbox(jload)
            data["object"]={"bbox":{"xmin":xmin,"xmax":xmax,"ymin":ymin,"ymax":ymax}}
            data["object"]["count"] = count
            
            final_json.append(data.copy())
            current_id+=1
            count_pos+=1
            
        #We deal with negatives
        else :
            keep = random.random()
            if (keep>=drop_neg_rate):
                with open(ann_path, 'r') as f:
                    jload = json.load(f)
                data["filename"]=img_path
                data["id"]=current_id
                data["height"]=jload["size"]["height"]
                data["width"]=jload["size"]["width"]
                data["format"]=img_path.split(".")[1]
                #no need to load bounding boxes
                data["object"]={"bbox":{}}
                data["object"]["count"]=0

                final_json.append(data.copy())
                current_id+=1
    
    
    
    with open('{}.json'.format(name), 'w') as f:
        f.write(json.dumps(final_json, indent=4))
    
    return print("Fichier final {}.json créé avec {} images positives et {} negatives \n \n \n \n \n \n".format(name,count_pos,current_id-count_pos))    

In [9]:
SAMPLES_PATHS = ['Images+json/']

all_paths = ReadPaths(SAMPLES_PATHS)
tr_paths, va_paths, te_paths = train_test_split(all_paths,0.9)

print(len(tr_paths))
print(len(va_paths))
print(len(te_paths))

create_json(va_paths,"valid",0)
create_json(te_paths,"test",0)
create_json(tr_paths,"train",0)


7638
424
425
['Images+json/img/monobloc (222)_OSKOmhfKbY.png', 'Images+json/ann/monobloc (222)_OSKOmhfKbY.json']
['Images+json/img/000000068765.jpg', 'Images+json/ann/000000068765.json']
['Images+json/img/000000178982.jpg', 'Images+json/ann/000000178982.json']
['Images+json/img/000000166563.jpg', 'Images+json/ann/000000166563.json']
['Images+json/img/000000006213.jpg', 'Images+json/ann/000000006213.json']
['Images+json/img/000000282037.jpg', 'Images+json/ann/000000282037.json']
['Images+json/img/000000359855.jpg', 'Images+json/ann/000000359855.json']
['Images+json/img/monobloc (213)_rUuGG15mxe.png', 'Images+json/ann/monobloc (213)_rUuGG15mxe.json']
['Images+json/img/000000151657.jpg', 'Images+json/ann/000000151657.json']
['Images+json/img/monobloc (12)_HoaJ5etbtz.jpg', 'Images+json/ann/monobloc (12)_HoaJ5etbtz.json']
['Images+json/img/monobloc (177)_gJW6ckfpCk.png', 'Images+json/ann/monobloc (177)_gJW6ckfpCk.json']
['Images+json/img/000000191288.jpg', 'Images+json/ann/000000191288.json

['Images+json/img/000000349860.jpg', 'Images+json/ann/000000349860.json']
['Images+json/img/monobloc (260)_zA83WRVJip.jpg', 'Images+json/ann/monobloc (260)_zA83WRVJip.json']
['Images+json/img/000000292060.jpg', 'Images+json/ann/000000292060.json']
['Images+json/img/000000231831.jpg', 'Images+json/ann/000000231831.json']
['Images+json/img/monobloc (235)_srHGQJHycW.png', 'Images+json/ann/monobloc (235)_srHGQJHycW.json']
['Images+json/img/000000456303.jpg', 'Images+json/ann/000000456303.json']
['Images+json/img/000000302536.jpg', 'Images+json/ann/000000302536.json']
['Images+json/img/000000462576.jpg', 'Images+json/ann/000000462576.json']
['Images+json/img/000000228942.jpg', 'Images+json/ann/000000228942.json']
['Images+json/img/000000226171.jpg', 'Images+json/ann/000000226171.json']
['Images+json/img/000000000724.jpg', 'Images+json/ann/000000000724.json']
['Images+json/img/bad (13).jpg', 'Images+json/ann/bad (13).json']
['Images+json/img/monobloc (101)_egbSmsW5B0.png', 'Images+json/ann/m

['Images+json/img/000000012667.jpg', 'Images+json/ann/000000012667.json']
['Images+json/img/000000157767.jpg', 'Images+json/ann/000000157767.json']
['Images+json/img/000000145597.jpg', 'Images+json/ann/000000145597.json']
['Images+json/img/monobloc (51)_NWkuVIpCDJ.png', 'Images+json/ann/monobloc (51)_NWkuVIpCDJ.json']
['Images+json/img/000000018837.jpg', 'Images+json/ann/000000018837.json']
['Images+json/img/monobloc (11)_RkeQHA6XLI.png', 'Images+json/ann/monobloc (11)_RkeQHA6XLI.json']
['Images+json/img/000000315001.jpg', 'Images+json/ann/000000315001.json']
['Images+json/img/000000229311.jpg', 'Images+json/ann/000000229311.json']
['Images+json/img/000000451714.jpg', 'Images+json/ann/000000451714.json']
['Images+json/img/monobloc (92)_jju09vNb4o.png', 'Images+json/ann/monobloc (92)_jju09vNb4o.json']
['Images+json/img/000000370677.jpg', 'Images+json/ann/000000370677.json']
['Images+json/img/000000180751.jpg', 'Images+json/ann/000000180751.json']
['Images+json/img/monobloc (299).png', '

['Images+json/img/monobloc (313)_RWvLszX3G9.png', 'Images+json/ann/monobloc (313)_RWvLszX3G9.json']
['Images+json/img/000000074200.jpg', 'Images+json/ann/000000074200.json']
['Images+json/img/monobloc (276)_JcAPqdhtwo.jpg', 'Images+json/ann/monobloc (276)_JcAPqdhtwo.json']
['Images+json/img/000000129945.jpg', 'Images+json/ann/000000129945.json']
['Images+json/img/000000155443.jpg', 'Images+json/ann/000000155443.json']
['Images+json/img/monobloc (315)_JRtPpMwwoZ.png', 'Images+json/ann/monobloc (315)_JRtPpMwwoZ.json']
['Images+json/img/000000079565.jpg', 'Images+json/ann/000000079565.json']
['Images+json/img/monobloc (171)_S5K7XeXYUk.png', 'Images+json/ann/monobloc (171)_S5K7XeXYUk.json']
['Images+json/img/000000350388.jpg', 'Images+json/ann/000000350388.json']
['Images+json/img/monobloc (254)_RtZaFknuFE.png', 'Images+json/ann/monobloc (254)_RtZaFknuFE.json']
['Images+json/img/monobloc (59)_zKhRlEkA8T.jpg', 'Images+json/ann/monobloc (59)_zKhRlEkA8T.json']
['Images+json/img/000000563267.j

['Images+json/img/monobloc (37).png', 'Images+json/ann/monobloc (37).json']
['Images+json/img/monobloc (305)_VgV1AMKENe.png', 'Images+json/ann/monobloc (305)_VgV1AMKENe.json']
['Images+json/img/monobloc (152)_urSW0lpRoa.png', 'Images+json/ann/monobloc (152)_urSW0lpRoa.json']
['Images+json/img/000000491008.jpg', 'Images+json/ann/000000491008.json']
['Images+json/img/000000204329.jpg', 'Images+json/ann/000000204329.json']
['Images+json/img/000000279278.jpg', 'Images+json/ann/000000279278.json']
['Images+json/img/000000155145.jpg', 'Images+json/ann/000000155145.json']
['Images+json/img/monobloc (32)_wCSoUvSnsl.png', 'Images+json/ann/monobloc (32)_wCSoUvSnsl.json']
['Images+json/img/000000099053.jpg', 'Images+json/ann/000000099053.json']
['Images+json/img/monobloc (12)_oneLRScYvz.png', 'Images+json/ann/monobloc (12)_oneLRScYvz.json']
['Images+json/img/monobloc (122)_L6vEuFzOdk.png', 'Images+json/ann/monobloc (122)_L6vEuFzOdk.json']
['Images+json/img/000000149568.jpg', 'Images+json/ann/0000

...

['Images+json/img/000000261161.jpg', 'Images+json/ann/000000261161.json']
['Images+json/img/monobloc (320)_K9Tg50uJil.png', 'Images+json/ann/monobloc (320)_K9Tg50uJil.json']
['Images+json/img/000000266892.jpg', 'Images+json/ann/000000266892.json']
['Images+json/img/000000001268.jpg', 'Images+json/ann/000000001268.json']
['Images+json/img/monobloc (155)_kpOi1K7BS6.png', 'Images+json/ann/monobloc (155)_kpOi1K7BS6.json']
['Images+json/img/monobloc (324)_8prlY2C0PI.png', 'Images+json/ann/monobloc (324)_8prlY2C0PI.json']
['Images+json/img/000000380203.jpg', 'Images+json/ann/000000380203.json']
['Images+json/img/monobloc (210)_A8kTFvbfEQ.jpg', 'Images+json/ann/monobloc (210)_A8kTFvbfEQ.json']
['Images+json/img/000000443303.jpg', 'Images+json/ann/000000443303.json']
['Images+json/img/monobloc (125)_KeOi2P7HIY.png', 'Images+json/ann/monobloc (125)_KeOi2P7HIY.json']
['Images+json/img/monobloc (70)_psRlm0zFEK.png', 'Images+json/ann/monobloc (70)_psRlm0zFEK.json']
['Images+json/img/000000292997.j

['Images+json/img/000000340451.jpg', 'Images+json/ann/000000340451.json']
['Images+json/img/monobloc (161)_UJm0Nc0ssL.png', 'Images+json/ann/monobloc (161)_UJm0Nc0ssL.json']
['Images+json/img/000000305317.jpg', 'Images+json/ann/000000305317.json']
['Images+json/img/000000571804.jpg', 'Images+json/ann/000000571804.json']
['Images+json/img/000000134882.jpg', 'Images+json/ann/000000134882.json']
['Images+json/img/000000226984.jpg', 'Images+json/ann/000000226984.json']
['Images+json/img/000000146358.jpg', 'Images+json/ann/000000146358.json']
['Images+json/img/monobloc (333)_mwakMFWYmr.png', 'Images+json/ann/monobloc (333)_mwakMFWYmr.json']
['Images+json/img/monobloc (321)_u6LE4PKXSV.jpg', 'Images+json/ann/monobloc (321)_u6LE4PKXSV.json']
['Images+json/img/monobloc (228)_c3MlJ0FUcH.png', 'Images+json/ann/monobloc (228)_c3MlJ0FUcH.json']
['Images+json/img/000000420916.jpg', 'Images+json/ann/000000420916.json']
['Images+json/img/monobloc (332)_JQ5b6DMJLi.png', 'Images+json/ann/monobloc (332)_

['Images+json/img/monobloc (336)_I66WfyVNio.png', 'Images+json/ann/monobloc (336)_I66WfyVNio.json']
['Images+json/img/monobloc (253)_T64IEz8HLb.png', 'Images+json/ann/monobloc (253)_T64IEz8HLb.json']
['Images+json/img/000000343976.jpg', 'Images+json/ann/000000343976.json']
['Images+json/img/000000404922.jpg', 'Images+json/ann/000000404922.json']
['Images+json/img/monobloc (89)_b5wYSyI3EY.png', 'Images+json/ann/monobloc (89)_b5wYSyI3EY.json']
['Images+json/img/000000205776.jpg', 'Images+json/ann/000000205776.json']
['Images+json/img/000000157365.jpg', 'Images+json/ann/000000157365.json']
['Images+json/img/000000261888.jpg', 'Images+json/ann/000000261888.json']
['Images+json/img/000000283785.jpg', 'Images+json/ann/000000283785.json']
['Images+json/img/monobloc (127)_kjZqt67Rzy.png', 'Images+json/ann/monobloc (127)_kjZqt67Rzy.json']
['Images+json/img/000000163258.jpg', 'Images+json/ann/000000163258.json']
['Images+json/img/monobloc (174).png', 'Images+json/ann/monobloc (174).json']
['Imag

['Images+json/img/000000234807.jpg', 'Images+json/ann/000000234807.json']
['Images+json/img/000000000285.jpg', 'Images+json/ann/000000000285.json']
['Images+json/img/000000084241.jpg', 'Images+json/ann/000000084241.json']
['Images+json/img/000000503823.jpg', 'Images+json/ann/000000503823.json']
['Images+json/img/000000408696.jpg', 'Images+json/ann/000000408696.json']
['Images+json/img/monobloc (226)_3cbrf7cXUu.png', 'Images+json/ann/monobloc (226)_3cbrf7cXUu.json']
['Images+json/img/000000442836.jpg', 'Images+json/ann/000000442836.json']
['Images+json/img/000000463849.jpg', 'Images+json/ann/000000463849.json']
['Images+json/img/monobloc (129)_tt9XGnqWhn.png', 'Images+json/ann/monobloc (129)_tt9XGnqWhn.json']
['Images+json/img/000000195165.jpg', 'Images+json/ann/000000195165.json']
['Images+json/img/000000407574.jpg', 'Images+json/ann/000000407574.json']
['Images+json/img/monobloc (34)_EhYH83FJyf.png', 'Images+json/ann/monobloc (34)_EhYH83FJyf.json']
['Images+json/img/000000154425.jpg',

Fichier final train.json créé avec 3154 images positives et 4484 negatives 
 
 
 
 
 



In [10]:
%run tfrecords-master/create_tfrecords.py \
--dataset_path train.json \
--prefix train \
--output_dir TFRecords/train \
--shards 1 \
--threads 1 \
--store_images

Launching 1 threads for spacings: [[0, 7638]]
2018-07-24 08:31:51.653701 [thread 0]: Processed 1000 of 7638 images in thread batch, with 0 errors.
2018-07-24 08:32:19.831880 [thread 0]: Processed 2000 of 7638 images in thread batch, with 0 errors.
2018-07-24 08:32:48.286401 [thread 0]: Processed 3000 of 7638 images in thread batch, with 0 errors.
2018-07-24 08:33:13.809720 [thread 0]: Processed 4000 of 7638 images in thread batch, with 0 errors.
2018-07-24 08:33:43.458177 [thread 0]: Processed 5000 of 7638 images in thread batch, with 0 errors.
2018-07-24 08:34:17.560285 [thread 0]: Processed 6000 of 7638 images in thread batch, with 0 errors.
2018-07-24 08:35:00.065601 [thread 0]: Processed 7000 of 7638 images in thread batch, with 0 errors.
2018-07-24 08:35:21.954290 [thread 0]: Wrote 7638 images to TFRecords/train/train-00000-of-00001.tfrecords, with 0 errors.
2018-07-24 08:35:21.958563 [thread 0]: Wrote 7638 images to 7638 shards, with 0 errors.
2018-07-24 08:35:22.146438: Finished

In [11]:
%run tfrecords-master/stat_tfrecords.py \
--stat 'verify_bboxes' \
--tfrecords "TFRecords/train/train-00000-of-00001.tfrecords"

Found 7638 images

Found 2 images with small bboxes

Found 0 images with reversed coordinates

Found 0 images with bbox count mismatches

Mean width: 144.8946
Median width: 95
Max width: 1954
Min width: 1

Mean height: 173.1926
Median height: 112
Max height: 2792
Min height: 1


In [12]:
%run tfrecords-master/create_tfrecords.py \
--dataset_path valid.json \
--prefix valid \
--output_dir TFRecords/valid \
--shards 1 \
--threads 1 \
--store_images

Launching 1 threads for spacings: [[0, 424]]
2018-07-24 08:36:00.997166 [thread 0]: Wrote 424 images to TFRecords/valid/valid-00000-of-00001.tfrecords, with 0 errors.
2018-07-24 08:36:01.005476 [thread 0]: Wrote 424 images to 424 shards, with 0 errors.
2018-07-24 08:36:01.431523: Finished writing all 424 images in data set.
0 examples failed.


In [13]:
%run tfrecords-master/create_tfrecords.py \
--dataset_path test.json \
--prefix test \
--output_dir TFRecords/test \
--shards 1 \
--threads 1 \
--store_images

Launching 1 threads for spacings: [[0, 425]]
2018-07-24 08:36:14.452473 [thread 0]: Wrote 425 images to TFRecords/test/test-00000-of-00001.tfrecords, with 0 errors.
2018-07-24 08:36:14.458519 [thread 0]: Wrote 425 images to 425 shards, with 0 errors.
2018-07-24 08:36:14.495002: Finished writing all 425 images in data set.
0 examples failed.


In [14]:
%run tfrecords-master/stat_tfrecords.py \
--stat 'verify_bboxes' \
--tfrecords "TFRecords/valid/valid-00000-of-00001.tfrecords"

Found 424 images

Found 0 images with small bboxes

Found 0 images with reversed coordinates

Found 0 images with bbox count mismatches

Mean width: 159.6132
Median width: 104
Max width: 1954
Min width: 6

Mean height: 201.5598
Median height: 130
Max height: 2792
Min height: 1


In [15]:
%run tfrecords-master/stat_tfrecords.py \
--stat 'verify_bboxes' \
--tfrecords "TFRecords/test/test-00000-of-00001.tfrecords"

Found 425 images

Found 0 images with small bboxes

Found 0 images with reversed coordinates

Found 0 images with bbox count mismatches

Mean width: 178.6904
Median width: 138
Max width: 1259
Min width: 7

Mean height: 211.6041
Median height: 164
Max height: 1026
Min height: 1
